In [ ]:
# 10/31/2021-1
# https://www.tensorflow.org/tutorials/load_data/images
from __future__ import absolute_import
from IPython.display import clear_output, display
import glob, os, shutil, sys, uuid
from os.path import *
from time import sleep
# from CV2_Utils import *

try: 
    from matplotlib import pyplot as plt
except Exception as err:
        %pip install matplotlib
        from matplotlib import pyplot as plt

try:
    pass
    import tensorflow as tf
except Exception as err:
    pass
    # print(err)
    %pip install tensorflow
    
import numpy as np

contentPath = os.getcwd()
tfImagesPath = join(contentPath, 'TensorflowImages')

import TarfileFunctions
from TarfileFunctions import *
from BashColors import C

if not exists(tfImagesPath):
    os.makedirs(tfImagesPath)
    os.chdir(tfImagesPath)
    try:
        tff.extractTarfiles('New_Juno_Original_Images.tar.gz')
    except FileNotFoundError as err:
        print(err)

os.chdir(contentPath)
from TarfileFunctions import *
from BashColors import C
tarfilePath=join(contentPath, 'New_Juno_Original_Images.tar.gz')
if exists(tarfilePath) and not exists(tfImagesPath):
    tff.extractTarfiles('New_Juno_Original_Images.tar.gz')

tfImagesPath = join(contentPath, 'TensorflowImages')

os.chdir(contentPath)
cv2Path = join(contentPath, 'CV2Images')

if not exists(cv2Path):
    os.mkdir(cv2Path)
    tff.extractTarfiles('All_CV2_Images.tar.gz')

os.chdir(contentPath)

print(f'tf version: {C.IBlue}{tf.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}{C.ColorOff}')


In [ ]:
import tensorflow as tf

In [ ]:
# image_dataset_from_directory function
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
help(image_dataset_from_directory)

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
if exists(cv2Path):
    print(cv2Path)
train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgba',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    batch_size=32)
print()
val_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgba',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    batch_size=32)

In [ ]:
class_names = train_ds.class_names
# print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(2):
    for i in range(3):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")


In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image),'-', np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=2)

In [ ]:
history = tf.keras.callbacks.History
# help(history)

In [ ]:
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    label = label_batch[i]
    plt.title(class_names[label])
    plt.axis("off")

In [ ]:
model_path=join(contentPath, 'first_model.h5')
model.save(model_path)
model.save_weights(model_path)

In [ ]:
print(model)
print(model.count_params())

print(model.input_shape)
print(model.run_eagerly)

In [ ]:
path=join(contentPath, 'enemy.png')
array=np.copy(path)

# Load the model
model = tf.keras.models.load_model(model_path,compile=True
# custom_objects=None,compile=True,options=None)

# try a sample
samples_to_predict=array


# Convert into Numpy array
samples_to_predict = np.array(samples_to_predict)

# Generate predictions for samples
predictions = model.predict(samples_to_predict)
print(predictions)